Download repository and dependencies for the abductive commonsense reasoning tasks.

In [ ]:
!git clone https://github.com/allenai/abductive-commonsense-reasoning.git
%cd abductive-commonsense-reasoning
!pip install -r requirements.txt
!sh get-data.sh

Cloning into 'abductive-commonsense-reasoning'...
remote: Enumerating objects: 261, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 261 (delta 3), reused 0 (delta 0), pack-reused 216 (from 1)
Receiving objects: 100% (261/261), 6.11 MiB | 35.17 MiB/s, done.
Resolving deltas: 100% (142/142), done.
/content/abductive-commonsense-reasoning/abductive-commonsense-reasoning
Obtaining pytorch-transformers from git+https://github.com/csbhagav/pytorch-transformers@generative-finetuning#egg=pytorch-transformers (from -r requirements.txt (line 20))
  Cloning https://github.com/csbhagav/pytorch-transformers (to revision generative-finetuning) to ./src/pytorch-transformers
  Running command git clone --filter=blob:none --quiet https://github.com/csbhagav/pytorch-transformers /content/abductive-commonsense-reasoning/abductive-commonsense-reasoning/src/pytorch-transformers


Exception in thread Thread-8 (target):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/oyama/oyama.py", line 33, in target
    raise Exception(f"Command exited with error code {rc}")
Exception: Command exited with error code -2


Output: Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIOsl+mE/BjcKhfdfnHspnc9v7+2F4SSNvm41Nlg1q+pB

2025/03/18 03:34:23 routes.go:1230: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:2048 OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 

In [ ]:
#!pip install -q git+https://github.com/HawkClaws/oyama.git ollama
from oyama import oyama
import ollama

model_path = "deepseek-r1:8b"
model_name = oyama.run(model_path)

command:=ollama --version
Output: Warning: could not connect to a running Ollama instance
command:=ollama serve
Server is not ready yet. Retrying...
Server is ready.
command:=ollama pull deepseek-r1:8b
Output: pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 6340dc3229b0... 100% ▕████████████████▏ 4.9 GB                         
pulling 369ca498f347... 100% ▕████████████████▏  387 B                         
pulling 6e4c38e1172f... 100% ▕████████████████▏ 1.1 KB                         
pulling f4d24e9138dd... 100% ▕████████████████▏  148 B                         
pulling 0cb05c6e4e02... 100% ▕████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 6340dc3229b0... 100% ▕████████████████▏ 4.9 GB                         
pulling 369ca498f347... 100% ▕████████████████▏  387 B                

In [ ]:
import pandas as pd

# 1. Load dev set
df_dev = pd.read_json("/content/abductive-commonsense-reasoning/abductive-commonsense-reasoning/data/anli/dev.jsonl", lines=True)  # each line is one JSON record
labels_dev = pd.read_csv("/content/abductive-commonsense-reasoning/abductive-commonsense-reasoning/data/anli/dev-labels.lst", header=None, names=["label"])
df_dev["label"] = labels_dev["label"]

# 2. Load test set
df_test = pd.read_json("/content/abductive-commonsense-reasoning/abductive-commonsense-reasoning/data/anli/test.jsonl", lines=True)
labels_test = pd.read_csv("/content/abductive-commonsense-reasoning/abductive-commonsense-reasoning/data/anli/test-labels.lst", header=None, names=["label"])
df_test["label"] = labels_test["label"]



def make_prompt(row):
    return (
        f"Given the following two observations, predict what occurred inbetween the observations:\n"
        f"obs 1. {row['obs1']}\n"
        f"obs 2. {row['obs2']}\n\n"
        f"Which hypothesis is most likely to have occurred between the observations?\n"
        f"hyp 1. {row['hyp1']}\n"
        f"hyp 2. {row['hyp2']}\n\n"
        "Please respond with ONLY the integer of the hypothesis (1 or 2)."
        "No explanation, no other text."
    )

df_test["prompt"] = df_test.apply(make_prompt, axis=1)
print(df_test["prompt"][0])


Given the following two observations, predict what occurred inbetween the observations:
obs 1. Jane was a professor teaching piano to students.
obs 2. Jane spent the morning sipping coffee and reading a book.

Which hypothesis is most likely to have occurred between the observations?
hyp 1. Two of Jane's students were early for their lessons.
hyp 2. None of Jane's students had a lesson that day.

Please respond with ONLY the integer of the hypothesis (1 or 2).


In [ ]:
import ollama
import pandas as pd
import concurrent.futures
import re

model_name = "deepseek-r1:8b"

def parse_digit(raw_text):
    matches = re.findall(r'\b([12])\b', raw_text)
    if not matches:
        return None
    return matches[-1]  # get the last occurrence

def query_model(prompt_text):
    response = ollama.chat(
        model=model_name,
        messages=[{"role": "user", "content": prompt_text}],
        stream=False
    )
    full_text = response.get("message", {}).get("content", "")
    extracted_digit = parse_digit(full_text)
    return extracted_digit, full_text

def run_inference_parallel(df, max_workers=4):
    df["prediction"] = None
    df["raw_response"] = None

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {}
        for idx, row in df.iterrows():
            prompt_text = row["prompt"]
            futures[executor.submit(query_model, prompt_text)] = idx

        for future in concurrent.futures.as_completed(futures):
            idx = futures[future]
            try:
                digit, raw = future.result()
                df.at[idx, "prediction"] = digit
                df.at[idx, "raw_response"] = raw
            except Exception as e:
                print(f"Error on row {idx}: {e}")
                df.at[idx, "prediction"] = None
                df.at[idx, "raw_response"] = ""
    return df

df_test_small = df_test.sample(n=500).copy()
anli_test_small_results = run_inference_parallel(df_test_small, max_workers=8)

In [ ]:
anli_test_small_results.to_csv("/content/sample_data/anli_test_small_results.csv", index=False)

In [ ]:
anli_test_small_results

,story_id,obs1,obs2,hyp1,hyp2,label,prompt,prediction,raw_response
1587,31dd9751-0538-404f-901d-48ce7e8fa86b-1,Kevin and Glenn's mother really loved musicals.,For her 50th birthday the boys surprised her w...,Kevin and Glenn's mother was about to turn 45.,they wanted to find her the best present.,2,"Given the following two observations, predict ...",2,"<think>\nOkay, so I need to figure out which h..."
695,29157c96-61c7-46d8-9ff2-12978b0b5e43-1,Bob loved to rock climb.,He fell to his death.,"One day, on a climb, Bob safety made the top.",Bob slipped and fell while climbing.,2,"Given the following two observations, predict ...",2,"<think>\nAlright, so I'm trying to figure out ..."
1392,aad0f120-f41a-4f05-9b62-955226ce3300-1,Pete was a quiet man.,"Pete shot the man in defense, and left him und...",Pete made a friend with a stranger who trespas...,A stranger tried to shoot Pete.,2,"Given the following two observations, predict ...",2,"<think>\nOkay, so I need to figure out which h..."
1998,66c79a45-25be-4066-9d36-016637bd57c52,Kev's dad asked to take him hunting.,Kev decided not to come hunting.,Kev hates animals and loves guns.,Kev loves animals and hates guns.,2,"Given the following two observations, predict ...",1,"<think>\nOkay, so I need to figure out what ha..."
2841,5b0a5d33-ad13-426b-9758-17387eeff03c-1,Anna could not swim.,Soon she was swimming quickly and expertly!,Anna took lessons at the pool.,"Anna was afraid of the water,.",1,"Given the following two observations, predict ...",1,"<think>\nOkay, so I need to figure out which h..."
...,...,...,...,...,...,...,...,...,...
2202,4a6a63bc-4a93-4e24-ae11-5e06a667e845-1,I got a bike for my sixth birthday.,That gave me enough momentum to balance and pe...,I got a flat tire on my bike.,My dad pushed me and let go.,2,"Given the following two observations, predict ...",2,"<think>\nOkay, so I've got this problem here w..."
1195,99cedf3e-db5f-41ea-96b0-de35c2cfad0b-1,"Late at night, May snuck out of her house.",Her town was beautiful in the moonlight!,She walked around and explored the forest.,May walked on the roads.,2,"Given the following two observations, predict ...",1,"<think>\nOkay, so I've got this problem here w..."
2565,c27472b1-6609-43b4-80ae-b56cb4b1995f-1,Rosie comes home late from a party and goes st...,She learns her lesson and vows never to neglec...,Rosie left her contacts in her eyes and got an...,Rosie didn't get enough food before the party.,1,"Given the following two observations, predict ...",1,"<think>\nOkay, so I'm trying to figure out wha..."
1892,4b471cd7-304b-400c-a06a-c4fa0ec9de93-1,Able was given a calf by his father.,Both Able and the calf had grown up but his fa...,Able said he would return when the calf was gr...,Abby's father than left.,2,"Given the following two observations, predict ...",2,"<think>\nOkay, so I'm trying to figure out wha..."


Calculating the accuracy of the deepseek distill model over 500 randomly sampled tests

In [ ]:

anli_test_small_results['label_str'] = anli_test_small_results['label'].astype(str)
anli_test_small_results['prediction_str'] = anli_test_small_results['prediction'].astype(str)

correct_mask = anli_test_small_results['label_str'] == anli_test_small_results['prediction_str']
num_correct = correct_mask.sum()
total = len(anli_test_small_results)
accuracy = num_correct / total

print(f"Correct: {num_correct} / {total}")
print(f"Accuracy: {accuracy:.2%}")

Correct: 383 / 500
Accuracy: 76.60%
